# Data Science E-commerce Project

## Coleta de Dados

### Introdução:

Este projeto tem como objetivo aplicar técnicas de Data Science para analisar e prever o comportamento de clientes em um e-commerce simulado, com foco em prever churn, recomendar produtos, realizar testes A/B e detectar fraudes. 

A fase de coleta de dados é crucial para criar um conjunto de dados inicial que servirá de base para todas as análises e modelos preditivos. Devido à ausência de dados reais, utilizaremos a geração de dados sintéticos para simular o comportamento de clientes, produtos e vendas.

A geração de dados sintéticos nos permitirá criar um ambiente de teste realista, facilitando o desenvolvimento e a validação dos modelos de análise. Além disso, essa abordagem nos dará flexibilidade para controlar a qualidade e a distribuição dos dados, garantindo que eles representem os padrões e características relevantes para o nosso e-commerce simulado.

### Configuração do Ambiente de Dados Sintéticos com Python:

Para gerar os dados sintéticos, configuraremos um ambiente de desenvolvimento Python no sistema operacional Windows, utilizando o VSCode como IDE. As seguintes bibliotecas serão utilizadas:

- **Faker**: Responsável por gerar dados fictícios realistas, como nomes, endereços, e-mails e outros atributos de clientes e produtos.
- **pandas**: Utilizada para manipular os dados gerados em formato de tabelas (DataFrames), facilitando a organização e o processamento dos dados.
- **numpy**: Fornece suporte para operações matemáticas e arrays multidimensionais, útil para manipulação de dados numéricos.
- **uuid**: Utilizada para gerar identificadores únicos universais (UUIDs), garantindo a unicidade dos registros nas tabelas.
- **random**: Fornece funções para geração de números aleatórios, útil para simular comportamentos aleatórios nos dados de vendas.
- **datetime**: Utilizada para manipular datas e horários, permitindo a geração de datas de vendas realistas.
- **sqlite3**: Utilizada para interagir com o banco de dados SQLite, permitindo a inserção dos dados gerados nas tabelas correspondentes.

O ambiente será configurado de forma a garantir a reprodutibilidade do processo de geração de dados, permitindo que os dados sejam gerados novamente, caso seja necessário aumentar o volume de dados ou gerar novos dados para testes.

Data Generation 🛠️🪄🏭

In [ ]:
from faker import Faker
import pandas as pd
import numpy as np
import uuid
import random
import matplotlib.pyplot as plt
from datetime import datetime
import sqlite3

fake = Faker('pt_BR')

conn = sqlite3.connect('salesEcommerce.db')
cursor = conn.cursor()

num_anos=5
base_clientes = 1500
taxa_crescimento_clientes = 0.2
vendas_por_cliente_ano = 2
taxa_crescimento_vendas = 0.25



categoriesDict = {"Eletronicos": ["Smartphone X", "Tablet Y", "Fone Bluetooth", "TV 4K", "Smartwatch", "Carregador Turbo", "Caixa de Som Bluetooth"],
                    "Computadores": ["Notebook Ultra", "PC Gamer", "Monitor Curvo", "Teclado Mecânico", "Mouse RGB", "Webcam Full HD", "SSD 1TB"],
                    "Roupas": ["Camiseta Dry Fit", "Jaqueta Jeans", "Tênis Running", "Vestido Casual", "Boné Esportivo", "Mochila Casual", "Óculos de Sol"],
                    "Livros": ["Python para Iniciantes", "Data Science Avançado", "O Poder do Hábito", "1984", "Mindset", "Clean Code", "A Arte da Guerra"],
                    "Beleza": ["Perfume Elegance", "Kit Skincare", "Batom Matte", "Shampoo Orgânico", "Máscara Facial", "Base Líquida", "Protetor Solar"],
                    "Automotivo": ["Pneu Aro 17", "Óleo Sintético", "Câmera de Ré", "Suporte Celular", "Capa para Banco", "Kit Ferramentas", "Lâmpada LED Automotiva"],
                    "Brinquedos": ["Carrinho Controle Remoto", "Boneca Fashion", "Lego Criativo", "Quebra-Cabeça 1000pçs", "Jogo Educativo", "Playset Cozinha", "Bola de Vinil"],
                    "Esportes": ["Bola de Futebol", "Raquete de Tênis", "Corda de Pular", "Mochila Esportiva", "Kit de Halteres", "Bicicleta Speed", "Luvas de Boxe"],
                    "Moveis": ["Sofá Retrátil", "Mesa de Jantar", "Cadeira Gamer", "Guarda-Roupa 6 Portas", "Cama Box Queen", "Estante de Livros", "Mesa de Escritório"],
                    "Eletrodomesticos": ["Geladeira Frost Free", "Micro-ondas Inox", "Máquina de Lavar", "Aspirador de Pó", "Fogão 5 Bocas", "Cafeteira Elétrica", "Liquidificador"],
                    "Ferramentas": ["Furadeira Elétrica", "Chave de Fenda", "Serra Circular", "Martelo Reforçado", "Trena Digital", "Alicate Universal", "Kit Brocas"],
                    "Petshop": ["Ração Premium", "Coleira Ajustável", "Brinquedo Interativo", "Cama para Cachorro", "Areia para Gato", "Shampoo para Pets", "Arranhador para Gatos"],
                    "Perfumaria": ["Desodorante Roll-on", "Hidratante Corporal", "Shampoo Anticaspa", "Sabonete Líquido", "Óleo Capilar", "Condicionador Nutritivo", "Creme para Mãos"],
                    "Papelaria": ["Caderno Universitário", "Caneta Esferográfica", "Marcador Permanente", "Papel Sulfite A4", "Planner Diário", "Grampeador", "Estojo Organizador"],
                    "Games": ["Console NextGen", "Controle Sem Fio", "Teclado Gamer RGB", "Headset Surround", "Cadeira Gamer Pro", "Mousepad XL", "Cartão Presente PSN"]}


Clientes

In [ ]:
def generate_customer(base_clientes, taxa_crescimento_clientes , num_anos):
    clientes_por_ano = [int(base_clientes * (1 + taxa_crescimento_clientes ) ** ano) for ano in range(num_anos)]
    todos_clientes = []
    info_clientes = {'customer_id': {}, 'address': {}, 'email': {}, 'birth_date': {}, 'state': {}}

    for ano, num_clientes in enumerate(clientes_por_ano):
        nomes_ano = [fake.name() for _ in range(num_clientes)]
        ids_ano = [str(uuid.uuid4())[:8] for _ in range(num_clientes)]
        endereco_ano = [fake.address() for _ in range(num_clientes)]
        emails_ano = [fake.email() for _ in range(num_clientes)]
        datas_nascimento_ano = [fake.date_of_birth(minimum_age=18, maximum_age=80) for _ in range(num_clientes)]
        state_ano = [fake.state() for _ in range(num_clientes)]

        for nome, id, endereco, email, data_nascimento, state in zip(nomes_ano, ids_ano, endereco_ano, emails_ano, datas_nascimento_ano, state_ano):
            todos_clientes.append(nome)
            info_clientes['customer_id'][nome] = id
            info_clientes['address'][nome] = endereco
            info_clientes['email'][nome] = email
            info_clientes['birth_date'][nome] = data_nascimento
            info_clientes['state'][nome] = state

    return todos_clientes, info_clientes


### Considerações Finais:

A qualidade dos dados sintéticos é crucial para a relevância das análises subsequentes. Dados que representam com precisão a distribuição e os padrões dos dados reais garantem que os modelos de machine learning e as análises exploratórias forneçam insights confiáveis.

Para alcançar essa representação, utilizamos a biblioteca Faker, que permite gerar dados fictícios com características realistas. Além disso, ajustamos os parâmetros de geração de dados para simular a distribuição de variáveis importantes, como preços de produtos e frequência de compras. A consistência entre as tabelas foi mantida através da geração de chaves primárias e estrangeiras, garantindo a integridade do banco de dados.

Este processo de geração de dados sintéticos pode ser repetido e ajustado conforme necessário, permitindo a criação de conjuntos de dados maiores ou a simulação de cenários específicos para testes adicionais.